In [ ]:
import networkx as nx

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

!apt-get install fonts-nanum*
!apt-get install fonts-woowa-hanna

In [ ]:
import matplotlib.font_manager as fm
NANUM_GOTHIC = fm.FontProperties(fname='/usr/share/fonts/truetype/nanum/NanumGothic.ttf')
NANUM_GOTHIC.get_name()

In [ ]:
import pandas as pd
import numpy as np
import os

import keras.layers as layers
import keras.optimizers as optimizers
from keras.models import Model, load_model
from keras import layers
from keras import models
from keras.models import load_model
import tensorflow as tf

import ipywidgets

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('font', family=NANUM_GOTHIC.get_name()) #한글 폰트 적용시

In [ ]:
#구글 드라이브에 연결
from google.colab import drive

drive.mount('/content/gdrive/')

In [ ]:
#작업 폴더 이동
os.chdir('/content/gdrive/My Drive/Colab Notebooks/urban-data-mining/08 Time Series/data/')
os.listdir()
# os.getcwd()

In [ ]:
#https://www.reb.or.kr/r-one/statistics/statisticsViewer.do?menuId=HOUSE_21111
#주택 매매가격 지수 부동산원
data = pd.read_excel('월간_매매가격지수_종합.xlsx', sheet_name=0, skiprows=10, header=[0,1])


data = data.loc[:, data.columns.get_level_values(0).isin({'지 역'}) | data.columns.get_level_values(1).isin({'지수'})]
data.columns = data.columns.get_level_values(0)
data.columns.values[0:4] = ['지역', '지역2', '지역3', '지역4']
data = data.loc[7:39] #서울만

data['지역']= data['지역'].fillna(method='ffill')
data['지역2']= data['지역2'].fillna(method='ffill')
data['지역3']= data['지역3'].fillna(method='ffill')

data = data.drop(data[data[data.columns[3]].isnull()].index)

data= data.fillna('')
data= data.replace('-', '0')


data['지역'] = data.apply(lambda x: x['지역'] + ' ' + x['지역2'] + ' ' + x['지역3'] + ' ' + x['지역4'], axis=1)
data = data.drop(['지역2', '지역3', '지역4'], axis=1)
data[data.columns[1:]] = data[data.columns[1:]].astype('float32')

data.shape

In [ ]:
TESTING_CNT = 9
TRAINING_MONTHS = 12

ndata = data[data.columns[1:]].values

ntraining = ndata[:, :-TESTING_CNT].T 
ntesting = ndata[:, -(TRAINING_MONTHS + TESTING_CNT):].T 

if len(ntraining.shape) == 1 :
  ntraining = ntraining.reshape([ntraining.shape[0], 1])
  ntesting = ntesting.reshape([ntesting.shape[0], 1])
  

print(ntesting.shape)

In [ ]:
data.columns

In [ ]:
data.iloc[1, 0]

In [ ]:
def display_graph(data_num = 0):
  ntraining = ndata[data_num, :-TESTING_CNT].T 
  ntesting = ndata[data_num,  -(TRAINING_MONTHS + TESTING_CNT):].T  

  if len(ntraining.shape) == 1 :
    ntraining = ntraining.reshape([ntraining.shape[0], 1])
    ntesting = ntesting.reshape([ntesting.shape[0], 1])
    

  plt.plot(np.arange(ntraining.shape[0]), ntraining)
  plt.plot(np.arange(ntesting.shape[0]) + ntraining.shape[0] - TRAINING_MONTHS, ntesting, color='red')
  plt.title(data.iloc[data_num, 0])
  plt.show()


ipywidgets.interact(display_graph, data_num=(0, data.shape[0], 1))

In [ ]:
def convert_to_matrix(data, step):
    x, y = [], []
    for i in range(len(data) - step):
        d = i + step
        x.append(data[i:d,:])
        y.append(data[d, :].reshape(1,data.shape[1]))
    return np.array(x), np.array(y)

ntraining_x, ntraining_y = convert_to_matrix(ntraining[:,:], TRAINING_MONTHS)
ntesting_x, ntesting_y = convert_to_matrix(ntesting[:,:], TRAINING_MONTHS)

print(ntraining_x.shape)
print(ntraining_y.shape)

print(ntesting_x.shape)
print(ntesting_y.shape)

In [ ]:
print(ntraining_x[1, :, 0])
print(ntraining_y[1, :, 0])

for seq in range(120,123) : 
  plt.plot(np.arange(ntraining_x.shape[1]), ntraining_x[seq, : , 0])
  plt.plot([ntraining_x.shape[1]-1, ntraining_x.shape[1]], [ntraining_x[seq,-1, 0], ntraining_y[seq, 0, 0]], color='red')
  plt.show()


In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

inputs = layers.Input(shape=(ntraining_x.shape[1], 1))
net = layers.BatchNormalization()(inputs)
net = layers.SimpleRNN(1, activation='relu')(net)
net = layers.Dense(1, activation='linear')(net)
model = Model(inputs, net)

model.compile(optimizer='adam', loss='mse')

model.summary()



In [ ]:
data_idx = 0

results = model.fit(ntraining_x[:,:,data_idx], ntraining_y[:,:,data_idx], epochs=300)

In [ ]:
pred_train = model.predict(ntraining_x[:,:,data_idx])
pred_test = model.predict(ntesting_x[:,:,data_idx])

In [ ]:
ntesting.shape[0]

In [ ]:
plt.figure(figsize=(15,10))

pre_n = ntraining.shape[0]
post_n = TESTING_CNT

plt.plot(np.arange(pre_n), ntraining[:, data_idx], color='black')
plt.plot(np.arange(pre_n-1, pre_n + TESTING_CNT), ntesting[-TESTING_CNT-1:, data_idx], color='blue')


plt.plot(np.arange(TRAINING_MONTHS, pre_n), pred_train, color='red')
plt.plot(np.arange(pre_n, pre_n + TESTING_CNT), pred_test[-TESTING_CNT:], color='red', linestyle='--')

# plt.legend(['input','real', 'predict'], loc='upper left')
plt.show()


In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

inputs = layers.Input(shape=(ntraining_x.shape[1], ntraining_x.shape[2]))
net = layers.BatchNormalization()(inputs)
net = layers.SimpleRNN(20, activation='relu')(net)
net = layers.Dense(50, activation='relu')(net)
net = layers.Dense(25)(net)
model = Model(inputs, net)

model.compile(optimizer='adam', loss='mse')

model.summary()





In [ ]:
results = model.fit(ntraining_x, ntraining_y.reshape(ntraining_y.shape[0],ntraining_y.shape[2]), epochs=300)

In [ ]:
pred_train = model.predict(ntraining_x)
pred_test = model.predict(ntesting_x)

In [ ]:
plt.figure(figsize=(15,10))

pre_n = ntraining.shape[0]
post_n = TESTING_CNT

plt.plot(ndata.T)
# plt.plot(np.arange(pre_n + TESTING_CNT), np.concatenate([ntraining[:, :], ntesting[-TESTING_CNT:, :]], axis=1))
# plt.plot(np.arange(pre_n-1, pre_n + TESTING_CNT), ntesting[-TESTING_CNT-1:, :], color='blue')


plt.plot(np.arange(TRAINING_MONTHS, pre_n), pred_train, color='#FF8888')
plt.plot(np.arange(pre_n, pre_n + TESTING_CNT), pred_test[-TESTING_CNT:], color='red', linestyle='--')

plt.show()


In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

inputs = layers.Input(shape=(ntraining_x.shape[1], ntraining_x.shape[2]))
net = layers.BatchNormalization()(inputs)
net = layers.LSTM(20, activation='relu')(net)
net = layers.Dense(50, activation='relu')(net)
net = layers.Dense(25)(net)
model = Model(inputs, net)

model.compile(optimizer='adam', loss='mse')

model.summary()

In [ ]:
results = model.fit(ntraining_x, ntraining_y.reshape(ntraining_y.shape[0],ntraining_y.shape[2]), epochs=300)

In [ ]:
pred_train = model.predict(ntraining_x)
pred_test = model.predict(ntesting_x)

In [ ]:
plt.figure(figsize=(15,10))

pre_n = ntraining.shape[0]
post_n = TESTING_CNT

plt.plot(ndata.T)
# plt.plot(np.arange(pre_n + TESTING_CNT), np.concatenate([ntraining[:, :], ntesting[-TESTING_CNT:, :]], axis=1))
# plt.plot(np.arange(pre_n-1, pre_n + TESTING_CNT), ntesting[-TESTING_CNT-1:, :], color='blue')


plt.plot(np.arange(TRAINING_MONTHS, pre_n), pred_train, color='#FF8888')
plt.plot(np.arange(pre_n, pre_n + TESTING_CNT), pred_test[-TESTING_CNT:], color='red', linestyle='--')

plt.show()
